![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [42]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

# Import any additional modules and start coding below
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rental_info = pd.read_csv('rental_info.csv')

#Rental Length in days
rental_info['rental_length_days'] = (pd.to_datetime(rental_info['return_date']) -        pd.to_datetime(rental_info['rental_date'])).dt.days

#Special Feature dummy variables
rental_info['deleted_scenes'] = np.where(rental_info['special_features'].str.contains('Deleted Scenes'),1,0)
rental_info['behind_the_scenes'] = np.where(rental_info['special_features'].str.contains('Behind the Scenes'),1,0)

#X and y
X = rental_info.drop(['rental_date', 'return_date', 'special_features', 'rental_length_days'], axis=1)
y = rental_info['rental_length_days']

#train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=9)

#Lasso Regression
lasso = Lasso(alpha=0.3, random_state=9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_
X_lasso_train, X_lasso_test = X_train.iloc[:,lasso_coef>0], X_test.iloc[:,lasso_coef>0]

#OLS models
ols = LinearRegression()
ols.fit(X_lasso_train, y_train)
y_pred_ols = ols.predict(X_lasso_test)
mse_lr = MSE(y_test, y_pred)

#Decision Tree
params_dt = {'min_samples_leaf': np.arange(0.1,0.9,1), 'max_depth':np.arange(1,11,1)}
dt = DecisionTreeRegressor()
rand_search_dt = RandomizedSearchCV(dt, param_distributions = params_dt, cv=5, random_state=9)
rand_search_dt.fit(X_train, y_train)
hyper_params_dt = rand_search_dt.best_params_
dt = DecisionTreeRegressor(min_samples_leaf=hyper_params_dt['min_samples_leaf'], max_depth=hyper_params_dt['max_depth'], random_state=9)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
mse_dt = MSE(y_test, y_pred_dt)

#Random Forest
param_rf = {'n_estimators': np.arange(1,101,1), 'max_depth': np.arange(1,11,1)}
rf = RandomForestRegressor()
rand_search_rf = RandomizedSearchCV(rf, param_distributions = param_rf, cv=5, random_state=9)
rand_search_rf.fit(X_train, y_train)
hyper_params_rf = rand_search_rf.best_params_
rf = RandomForestRegressor(n_estimators = hyper_params_rf['n_estimators'], max_depth=hyper_params_rf['max_depth'], random_state=9)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mse_rf = MSE(y_test, y_pred_rf)

print(mse_lr, mse_tree, mse_rf)

best_model = rf
best_mse = mse_rf

4.812297241276244 3.645633359745855 2.225667528098759
